In [1]:
import geopandas as gpd
import pandas as pd
import folium
from siuba import *

import utils

In [2]:
import os

In [3]:
ti_parcels = gpd.read_file('ti_ybi_parcels_2024-11-03.geojsonl')

In [4]:
# ti_parcels.explore(column='is_street')

In [5]:
m = ti_parcels.explore(column='open_space')

In [6]:
gdf2 = gpd.read_file('source_data/ti_gaa_north.geojson')

In [7]:
gdf3 = gpd.read_file('source_data/palms_2.geojson') >> filter(_.open_space == 'active')

In [8]:
gdf = pd.concat([ti_parcels, gdf2, gdf3])

In [9]:
gdf = gdf.to_crs('EPSG:3310')

In [10]:
gdf.geometry = gdf.geometry.buffer(-2)

In [11]:
import branca

In [12]:
branca.colormap.step.Greens_09

In [13]:
[branca.colormap.step.Greens_09.rgb_hex_str(x) for x in branca.colormap.step.Greens_09.index]

['#f7fcf5',
 '#e5f5e0',
 '#c7e9c0',
 '#a1d99b',
 '#74c476',
 '#41ab5d',
 '#238b45',
 '#006d2c',
 '#00441b',
 '#00441b']

In [14]:
gdf['parcel_type'] = gdf.apply(lambda x: x.open_space + ' open space' if x.open_space else 'street' if x.is_street else 'ordinary', axis=1)

In [15]:
color_dict = {'street': 'red', 'ordinary': 'gray', 'active': '#00441b', 'planned': '#74c476'}

In [16]:
color_list = ['#00441b', 'gray', '#74c476', 'black']

In [17]:
import xyzservices.providers as xyz

#Add the Stadia Maps Stamen Toner provider details via xyzservices
tile_provider = xyz.Stadia.StamenTonerLite

#Update the URL to include the API key placeholder
tile_provider["url"] = tile_provider["url"] + f"?api_key={os.environ['STADIA_API_KEY']}"

#Create the folium TileLayer, specifying the API key
stamen = folium.TileLayer(
    tiles=tile_provider.build_url(api_key=os.environ["STADIA_API_KEY"]),
    attr=tile_provider.attribution,
    name=tile_provider.name,
    max_zoom=tile_provider.max_zoom,
    detect_retina=True
)

In [18]:
m = gdf.explore(column = 'parcel_type', cmap = color_list, tiles = stamen,
           style_kwds={'opacity': 0})

In [22]:
plugins.Fullscreen(
    position="topright",
    title="Expand me",
    title_cancel="Exit me",
    force_separate_button=True,
).add_to(m)

In [23]:
m

In [20]:
folium.__version__

'0.18.0'

In [21]:
from folium import plugins